In [102]:
# Install neccesary packages 

import pandas as pd 
import numpy as np 
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

In [60]:
# Read file 
data = pd.read_csv("/Users/venketeshprasathmanoharan/Desktop/word_count/Top_250_Movie.csv")

In [61]:
# Creating a column Views to update the given value 

data["views"] = ""
data.columns

Index(['ID', 'Title', 'Year', 'Rating ', 'Rating_count', 'views'], dtype='object')

In [62]:
# Update the given values to the new columns \

data.loc[data['Title'] == 'Forest Gump', 'views'] = 10000000
data.loc[data['Title'] == 'The Usual Suspects', 'views'] = 7500000
data.loc[data['Title'] == 'Rear Window', 'views'] = 6000000
data.loc[data['Title'] == 'North by Northwest', 'views'] = 4000000
data.loc[data['Title'] == 'The Secret in Their Eyes', 'views'] = 3000000
data.loc[data['Title'] == 'Spotlight', 'views'] = 1000000


In [63]:
# Preprocessing the data 


df = data.replace(',', '', regex = True) # Removing the commas to make str object to int
nan_value = float("NaN")
df.replace("", nan_value, inplace=True) # Replacing Nan values to empty cells 

In [64]:
# Create a seperate Dataframe to store values which has Nan

non_nan_df = df[df.isna().any(axis=1)] # Nan dataframe to predict

# Excluding the Nan values 
new = df.dropna() 

In [65]:
y = new['views']  # Dependant Variable  
x = new.drop(['ID', 'Title', 'Year', 'views'], axis=1) # Independant variables


In [66]:
# changing type to numeric 
x = x.apply(pd.to_numeric, errors='ignore')

In [ ]:
# applying linear regression 
reg = LinearRegression().fit(x, y)

xg_reg = xgb.XGBRegressor(objective ='reg:linear', n_estimators=1)
xg_reg.fit(x,y)

regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(x, y)

In [92]:
# Storing id of movies to estimate it views 
Id_list = non_nan_df['ID'].to_list()


df_pred = pd.DataFrame(columns = ['ID','Title','Year', 'Rating', 'Rating_count', 'Views'])
for i in Id_list:
    test1 = non_nan_df[(non_nan_df['ID'] == i)]
    test = test1[['Rating ', 'Rating_count']].apply(pd.to_numeric, errors='ignore')
    test1['views'] = int(xg_reg.predict(test))
    df_pred.loc[i] = test1.values[0]


/var/folders/k8/n2zsd8r164l74hwq2bvv9q8m0000gp/T/ipykernel_39166/1866010085.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test1['views'] = int(xg_reg.predict(test))
